### Exploratory Analysis Overview

This notebook documents the **exploratory phase** of the ESDP final project.

The purpose of this notebook is to inspect and understand the structure, spatial and temporal resolution, units, and coordinate conventions of two monthly gridded precipitation datasets that will later be compared over Northern India.

The datasets explored here are:

#### IMERG Final Run (Version 07, Monthly)
- Integrated Multi-satellitE Retrievals for GPM  
- Monthly Final Run product  
- Native spatial resolution: 0.1° × 0.1°  
- Global coverage with longitude convention −180° to 180°  
- Precipitation provided in mm hr⁻¹  

#### GPCP Monthly Precipitation (Version 2.3)
- Global Precipitation Climatology Project merged satellite–gauge product  
- Monthly precipitation dataset  
- Native spatial resolution: 2.5° × 2.5°  
- Global coverage with longitude convention 0° to 360°  
- Precipitation provided in mm day⁻¹  

This notebook is intentionally **exploratory and diagnostic**.  
All final data processing steps, including spatial subsetting, unit conversion, longitude normalization, regridding, and the generation of comparison-ready datasets, are implemented in standalone scripts under the `src/` directory.


In [1]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
import xarray as xr
import h5py
import requests
pd.set_option('display.max_rows', 100)


In [2]:
raw_dir = Path("../data/raw/imerg_monthly")
raw_dir.mkdir(parents=True, exist_ok=True)

raw_dir

WindowsPath('../data/raw/imerg_monthly')

#### Inspecting Available IMERG Monthly Files on NASA GES DISC

Before attempting automated downloads of IMERG monthly products, it is useful to first verify what files are available on the NASA GES DISC server for a given year.

The function below builds the URL for the IMERG Final Run monthly data (Version 07) corresponding to the specified year. It sends a simple HTTP request to the server directory, parses the returned HTML using BeautifulSoup, and collects all links that end with the `.HDF5` extension.

This approach allows us to:
- confirm the file naming convention used by the IMERG monthly product,
- check that the files we expect are present for a given year, and
- avoid attempting to download missing or incorrectly named files.

For a more complete example of how IMERG downloads are implemented in practice, including authentication and iterative download logic, have a look at this notebook:  
https://github.com/gsgauravsomani/earth-system-data-processing/blob/main/data_access/load_nasa_imerg.ipynb

The output of the function is a list of IMERG monthly filenames available for the selected year.

In [3]:
from bs4 import BeautifulSoup

def list_imerg_monthly_files(year):
    url = f"https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGM.07/{year}/"
    
    r = requests.get(url)
    r.raise_for_status()
    
    soup = BeautifulSoup(r.text, "html.parser")
    
    files = [
        link.get("href")
        for link in soup.find_all("a")
        if link.get("href", "").endswith(".HDF5")
    ]
    
    return files


# Test for one year
list_imerg_monthly_files(2019)[:5]

['3B-MO.MS.MRG.3IMERG.20190101-S000000-E235959.01.V07B.HDF5',
 '3B-MO.MS.MRG.3IMERG.20190101-S000000-E235959.01.V07B.HDF5',
 '3B-MO.MS.MRG.3IMERG.20190201-S000000-E235959.02.V07B.HDF5',
 '3B-MO.MS.MRG.3IMERG.20190201-S000000-E235959.02.V07B.HDF5',
 '3B-MO.MS.MRG.3IMERG.20190301-S000000-E235959.03.V07B.HDF5']

#### Downloading IMERG Monthly Data (2019–2021)

For this project, I focus on a three-year period from **2019 to 2021** to perform a climatological comparison between IMERG and GPCP precipitation data.

The code below uses the previously defined helper function to list all available IMERG Final Run monthly files for each year in this period and download them from the NASA GES DISC server. Access to the data is handled using a local `.netrc` file for authentication.

Files that are already present locally are skipped, so the download process can be safely re-run without duplicating data. The downloaded files are stored in the raw data directory and are later processed using the standalone scripts in the `src/` folder.

In [5]:
base_url = "https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGM.07"
session = requests.Session()
session.trust_env = True  # uses .netrc

downloaded_files = []

for year in range(2019, 2022):
    try:
        files = list_imerg_monthly_files(year)
    except Exception as e:
        print(f"Could not list files for {year}: {e}")
        continue

    for fname in files:
        url = f"{base_url}/{year}/{fname}"
        out_file = raw_dir / fname

        if out_file.exists():
            downloaded_files.append(out_file)
            continue

        try:
            r = session.get(url, stream=True, timeout=60)
            r.raise_for_status()

            with open(out_file, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)

            downloaded_files.append(out_file)

        except Exception as e:
            print(f"Skipping {fname}: {e}")


In [7]:
# Local data directory (already downloaded IMERG monthly files)
raw_dir = Path('../data/raw/imerg_monthly')
files = sorted(raw_dir.glob('*.HDF5'))

print(f'Found {len(files)} files in {raw_dir.resolve()}')
print('First file:', files[0].name if files else 'None')
print('Last file :', files[-1].name if files else 'None')

Found 36 files in D:\esdp\ESDP-final-project\data\raw\imerg_monthly
First file: 3B-MO.MS.MRG.3IMERG.20190101-S000000-E235959.01.V07B.HDF5
Last file : 3B-MO.MS.MRG.3IMERG.20211201-S000000-E235959.12.V07B.HDF5


In [8]:
# Parse month from filename and check for gaps
def parse_month_from_name(name: str) -> pd.Timestamp:
    m = re.search(r'\.(\d{8})-S', name)
    if not m:
        raise ValueError(f'Cannot parse date from {name}')
    return pd.to_datetime(m.group(1), format='%Y%m%d')

file_months = pd.DatetimeIndex([parse_month_from_name(p.name) for p in files])
expected = pd.date_range(file_months.min(), file_months.max(), freq='MS')
missing = expected.difference(file_months)

print('Date span:', file_months.min().date(), 'to', file_months.max().date())
print('Expected months:', len(expected))
print('Available months:', len(file_months))
print('Missing months:', len(missing))
print(missing.tolist() if len(missing) else 'None')


Date span: 2019-01-01 to 2021-12-01
Expected months: 36
Available months: 36
Missing months: 0
None


In [9]:
# Inspect one sample HDF5 file structure
sample = files[0]
print('Sample file:', sample.name)

with h5py.File(sample, 'r') as f:
    datasets = []
    def walk(name, obj):
        if isinstance(obj, h5py.Dataset):
            datasets.append((name, obj.shape, str(obj.dtype)))
    f.visititems(walk)

for name, shape, dtype in datasets:
    print(f'{name:45s} shape={str(shape):18s} dtype={dtype}')


Sample file: 3B-MO.MS.MRG.3IMERG.20190101-S000000-E235959.01.V07B.HDF5
Grid/gaugeRelativeWeighting                   shape=(1, 3600, 1800)    dtype=int16
Grid/lat                                      shape=(1800,)            dtype=float32
Grid/lat_bnds                                 shape=(1800, 2)          dtype=float32
Grid/latv                                     shape=(2,)               dtype=int32
Grid/lon                                      shape=(3600,)            dtype=float32
Grid/lon_bnds                                 shape=(3600, 2)          dtype=float32
Grid/lonv                                     shape=(2,)               dtype=int32
Grid/nv                                       shape=(2,)               dtype=int32
Grid/precipitation                            shape=(1, 3600, 1800)    dtype=float32
Grid/precipitationQualityIndex                shape=(1, 3600, 1800)    dtype=float32
Grid/probabilityLiquidPrecipitation           shape=(1, 3600, 1800)    dtype=int16
Grid

#### Inspecting a Single IMERG Monthly File

To understand the structure of the IMERG monthly dataset, a single file
is opened and inspected for variables, coordinates, units, and grid layout.


In [10]:
# Open one sample with xarray (IMERG science variables live in group='Grid')
ds1 = xr.open_dataset(sample, engine='netcdf4', group='Grid')

print(ds1)
print('Variables:', list(ds1.data_vars))
print('Coordinates:', list(ds1.coords))
print('Precip units:', ds1['precipitation'].attrs.get('units'))

ds1.close()


<xarray.Dataset> Size: 130MB
Dimensions:                         (time: 1, lon: 3600, lat: 1800, nv: 2,
                                     lonv: 2, latv: 2)
Coordinates:
  * time                            (time) object 8B 2019-01-01 00:00:00
  * lon                             (lon) float32 14kB -179.9 -179.9 ... 179.9
  * lat                             (lat) float32 7kB -89.95 -89.85 ... 89.95
Dimensions without coordinates: nv, lonv, latv
Data variables:
    time_bnds                       (time, nv) object 16B ...
    lon_bnds                        (lon, lonv) float32 29kB ...
    lat_bnds                        (lat, latv) float32 14kB ...
    precipitation                   (time, lon, lat) float32 26MB ...
    randomError                     (time, lon, lat) float32 26MB ...
    gaugeRelativeWeighting          (time, lon, lat) float32 26MB ...
    probabilityLiquidPrecipitation  (time, lon, lat) float32 26MB ...
    precipitationQualityIndex       (time, lon, lat) float32 26

The IMERG monthly files are concatenated here only to verify temporal
continuity across the selected period.

This step is exploratory and no processed output is saved.
The full concatenation and subsetting will be implemented in scripts.


In [11]:
imerg = xr.open_mfdataset(
    files,
    engine="netcdf4",
    group="Grid",
    combine="by_coords",
    data_vars="minimal",
    coords="minimal",
    compat="override",
)

pr_imerg = imerg["precipitation"].transpose("time", "lat", "lon")
pr_imerg

<xarray.DataArray 'precipitation' (time: 36, lat: 1800, lon: 3600)> Size: 933MB
dask.array<transpose, shape=(36, 1800, 3600), dtype=float32, chunksize=(1, 1800, 145), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 288B 2019-01-01 00:00:00 ... 2021-12-01 00:00:00
  * lon      (lon) float32 14kB -179.9 -179.9 -179.8 ... 179.8 179.8 179.9
  * lat      (lat) float32 7kB -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
Attributes:
    DimensionNames:    time,lon,lat
    Units:             mm/hr
    units:             mm/hr
    CodeMissingValue:  -9999.9
    LongName:          Merged microwave-infrared-gauge precipitation estimate

#### IMERG Exploratory Diagnostics

The checks below verify temporal continuity and basic value ranges
before any scripted processing is treated as production-ready.


In [12]:
print('IMERG dimensions:', dict(pr_imerg.sizes))
print('IMERG time span :', str(pr_imerg.time.values[0])[:10], 'to', str(pr_imerg.time.values[-1])[:10])
print('IMERG lat range :', float(pr_imerg.lat.min()), 'to', float(pr_imerg.lat.max()))
print('IMERG lon range :', float(pr_imerg.lon.min()), 'to', float(pr_imerg.lon.max()))
print('IMERG units     :', pr_imerg.attrs.get('units', 'NA'))
print('IMERG min/max   :', float(pr_imerg.min()), '/', float(pr_imerg.max()))


IMERG dimensions: {'time': 36, 'lat': 1800, 'lon': 3600}
IMERG time span : 2019-01-01 to 2021-12-01
IMERG lat range : -89.94999694824219 to 89.94999694824219
IMERG lon range : -179.9499969482422 to 179.9499969482422
IMERG units     : mm/hr
IMERG min/max   : 0.0 / 4.550000190734863


#### GPCP Monthly Precipitation Dataset

For 2019-2021, GPCP monthly files (36 total) are available in
`../data/raw/gpcp_monthly`. The files were manually downloaded from the data archives available at.

The cells below inspect GPCP explicitly (dimensions, coordinates, units,
latitude/longitude conventions, and time coverage) so later harmonization
choices are traceable.


In [13]:
ds_gpcp = xr.open_dataset("../data/raw/gpcp_monthly/gpcp_v02r03_monthly_d201901_c20190407.nc")
ds_gpcp

<xarray.Dataset> Size: 86kB
Dimensions:       (latitude: 72, longitude: 144, time: 1, nv: 2)
Coordinates:
  * latitude      (latitude) float32 288B -88.75 -86.25 -83.75 ... 86.25 88.75
  * longitude     (longitude) float32 576B 1.25 3.75 6.25 ... 353.8 356.2 358.8
  * time          (time) datetime64[ns] 8B 2019-01-01
Dimensions without coordinates: nv
Data variables:
    lat_bounds    (latitude, nv) float32 576B ...
    lon_bounds    (longitude, nv) float32 1kB ...
    time_bounds   (time, nv) datetime64[ns] 16B ...
    precip        (time, latitude, longitude) float32 41kB ...
    precip_error  (time, latitude, longitude) float32 41kB ...
Attributes: (12/45)
    Conventions:                CF-1.6, ACDD 1.3
    title:                      Global Precipitation Climatatology Project (G...
    source:                     oc.201901.sg
    references:                 Huffman et al. 1997, http://dx.doi.org/10.117...
    history:                    1) `�-��, Dr. Jian-Jian Wang, U of Maryland,...
    Metadata_Conventions:       CF-1.6, Unidata Dataset Discovery v1.0, NOAA ...
    ...                         ...
    metadata_link:              gov.noaa.ncdc:C00979
    product_version:            v23rB1
    platform:                   GOES (Geostationary Operational Environmental...
    sensor:                     Imager, Imager, Imager, AIRS > Atmospheric In...
    spatial_resolution:         2.5 degree
    comment:                    Processing computer: eagle2.umd.edu

#### GPCP Exploratory Diagnostics

These checks make GPCP assumptions explicit before processing:
coordinate names, axis direction, units, and monthly file continuity.


In [14]:
gpcp_files = sorted(Path('../data/raw/gpcp_monthly').glob('gpcp_v02r03_monthly_*.nc'))
print('GPCP file count:', len(gpcp_files))

print('GPCP dims      :', dict(ds_gpcp.sizes))
print('GPCP coords    :', list(ds_gpcp.coords))
print('GPCP variables :', list(ds_gpcp.data_vars))
print('Precip units   :', ds_gpcp['precip'].attrs.get('units', 'NA'))

lat0, lat1 = float(ds_gpcp.latitude.values[0]), float(ds_gpcp.latitude.values[-1])
lon0, lon1 = float(ds_gpcp.longitude.values[0]), float(ds_gpcp.longitude.values[-1])
lat_order = 'ascending' if lat0 < lat1 else 'descending'
print('Latitude range :', lat0, 'to', lat1, f'({lat_order})')
print('Longitude range:', lon0, 'to', lon1)

times = []
for fp in gpcp_files:
    with xr.open_dataset(fp) as d:
        times.append(pd.to_datetime(d.time.values[0]))
times = pd.DatetimeIndex(times).sort_values()
expected = pd.date_range(times.min(), times.max(), freq='MS')
missing = expected.difference(times)
print('Time span      :', times.min().date(), 'to', times.max().date())
print('Missing months :', len(missing), list(missing.date) if len(missing) else 'None')


GPCP file count: 36
GPCP dims      : {'latitude': 72, 'longitude': 144, 'time': 1, 'nv': 2}
GPCP coords    : ['latitude', 'longitude', 'time']
GPCP variables : ['lat_bounds', 'lon_bounds', 'time_bounds', 'precip', 'precip_error']
Precip units   : mm/day
Latitude range : -88.75 to 88.75 (ascending)
Longitude range: 1.25 to 358.75
Time span      : 2019-01-01 to 2021-12-01
Missing months : 0 None


#### Side-by-Side Metadata Snapshot

A compact comparison helps confirm what must be harmonized later
(resolution, coordinate names, and longitude convention).


In [15]:
comparison = pd.DataFrame([
    {
        'dataset': 'IMERG',
        'time_steps': int(pr_imerg.sizes['time']),
        'lat_points': int(pr_imerg.sizes['lat']),
        'lon_points': int(pr_imerg.sizes['lon']),
        'units': pr_imerg.attrs.get('units', 'NA'),
        'lon_min': float(pr_imerg.lon.min()),
        'lon_max': float(pr_imerg.lon.max()),
    },
    {
        'dataset': 'GPCP',
        'time_steps': int(ds_gpcp.sizes['time']),
        'lat_points': int(ds_gpcp.sizes['latitude']),
        'lon_points': int(ds_gpcp.sizes['longitude']),
        'units': ds_gpcp['precip'].attrs.get('units', 'NA'),
        'lon_min': float(ds_gpcp.longitude.min()),
        'lon_max': float(ds_gpcp.longitude.max()),
    },
])
comparison


,dataset,time_steps,lat_points,lon_points,units,lon_min,lon_max
0,IMERG,36,1800,3600,mm/hr,-179.949997,179.949997
1,GPCP,1,72,144,mm/day,1.250000,358.750000


### What We Learned From the Exploratory Analysis

This exploratory notebook helped clarify several important differences between the IMERG and GPCP precipitation products that directly shape the final processing pipeline.

Although both datasets provide **monthly precipitation**, they differ substantially in how that information is represented:

- **Spatial resolution**  
  - IMERG: high-resolution (0.1°) global grid  
  - GPCP: coarse resolution (2.5° × 2.5°)

- **Longitude conventions**  
  - IMERG uses a **−180° to 180°** longitude range  
  - GPCP uses a **0° to 360°** longitude range  
  This mismatch requires an explicit longitude transformation before any comparison.

- **Coordinate naming and ordering**  
  - IMERG uses `lat` / `lon` with increasing latitude  
  - GPCP uses `latitude` / `longitude`, with latitude stored in descending order  
  These differences must be normalized to avoid silent subsetting or interpolation errors.

- **Units and meaning of precipitation**  
  - IMERG monthly files store precipitation as **rates (mm/hr)**  
  - GPCP stores **monthly accumulated precipitation (mm/day equivalent)**  
  As a result, IMERG must be converted to compatible units before regridding.

- **Time representation**  
  - Both datasets are monthly, but differ in how time bounds and metadata are encoded  
  Ensuring consistent `datetime64` handling is necessary for alignment and validation.

Most importantly, this notebook showed that **direct comparison without preprocessing would be incorrect**, even though both datasets appear similar at first glance.

### Project Structure
Based on the exploratory analysis in this notebook, the following workflow seems appropriate:

- `data/raw/`
  - `imerg_monthly/` – IMERG monthly Final Run files (2019–2021)
  - `gpcp_monthly/` – GPCP v2.3 monthly files (manually downloaded)

- `data/processed/`
  - `imerg_north_india.nc`
  - `imerg_north_india_mmday.nc`
  - `gpcp_north_india.nc`
  - `imerg_north_india_on_gpcp_grid.nc`
  - `regrid_sanity_check_report.json`

- `src/`
  - `config.py` – domain, time range, and path configuration
  - `concatenate_imerg.py`
  - `unit_convert_imerg.py`
  - `extract_gpcp.py`
  - `regrid_imerg_to_gpcp.py`
  - `sanity_check_regrid.py`
  - `run_pipeline.py`

- `notebooks/`
  - `Exploratory-Notebook.ipynb` – this notebook (exploration only)